<a href="https://colab.research.google.com/github/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Simplificação de Texto

Nas primeiras abordagens, houve uma tentativa de produzir uma rede neural que tivesse como entrada o *token frequence* e saida outro *token frequence*. Apesar disso poder ser considerado um resultado válido para a tentativa de melhorar a classificação de texto, não podemos considerar uma frase como um conjunto de palavras com a propriedade da comutatividade. A ordem das palavras acaba importando muito para os humanos.

## importando bugingangas

In [0]:
import numpy as np
import pandas as pd

## Paper With Code

~~Após um gole de sorte, eu acabo por encontrar essa maravilha~~ chamado de [Paper With Code](https://paperswithcode.com/sota/document-summarization-on-cnn-daily-mail), é um ~~bom~~ compilado de resultados cientificos sobre determinados problemas da computação. O primeiro artigo do link de cima, mostra uma solução com ROGUE-1 de 43.83 para o problema de sumarização de documentos como o *state of the art*.

![paper with code](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/paperwithcode.png?raw=true)
<center>paper with code</center>

No artigo *Text Summarization with Pretrained Encoders* (LIU e LAPATA, 2019), os autores representam o texto como *Bert*, que é gerado por uma rede neural e é um caso especial do *word2vec*.




## Representação do Bert word2vec

*Kyubyong* criou um preset do Bert, um dicionário onde cada palavra é representada por um vetor de 768 valores decimais. Visto algumas limitações das plataformas *Google Colab* e do *Github*, serão usadas apenas 300 instâncias do CNN, que gerou um dicionário reduzido de apenas 25mb.

![Bert is Evil](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/audio-banner.jpg?raw=true)
<center>Bert is Evil</center>

## Base de dados CNN

A base de dados pode ser encontrado [nesse github da google](https://github.com/google-research-datasets/sentence-compression), no formato *Json*. Essa base de dados é formada com notícias da CNN com a primeira linha da noticia, com o título, com todos os bigramas possíveis e com informações do NER. Porém no trabalho apenas será usado a primeira linha como entrada e o título como saída.

![Json](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/Jasonf.jpg?raw=true)
<center>Json</center>

## Abrindo uma noticia (bert) e convertendo para texto.

In [4]:
# Abrindo o dicionario
dicurl = 'https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/dataset/dicionario.csv'
dicio = pd.read_csv(dicurl)

#abrindo a noticia
nn = input("digite um numero entre 0 e 199: ")



ParserError: ignored

## Referencias

